In [17]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import ta
# from datapackage import Package
# from selenium import webdriver
# import requests
# from bs4 import BeautifulSoup
# import yfinance as yf
import os
import random
import shutil
import time
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from tensorflow.keras.utils import plot_model
import graphviz
import pydot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

In [18]:
# PARAMETERS

steps = 20
epochs = 50
nodes_per_layer = 64
shift = -1
batch_size = 20
loss_function = "mean_squared_error"
dropout_rate = 0.2

In [19]:
start_time = time.time()

for file in os.listdir("data"):
	ticker_code = file.replace(".csv", "")
	dataset = pd.read_csv("data/" + file)
	shift = -1
	dataset = dataset.drop(columns=["Date", "Dividends", "Stock Splits"])
	train_set_len = int(len(dataset) * 0.8)
	train_set = dataset[:train_set_len]
	test_set = dataset[train_set_len:]
	X_train = train_set.drop(columns=["Close"])[:shift]
	y_train = train_set["Close"].shift(shift).dropna()
	X_test = test_set.drop(columns=["Close"])[:shift]
	y_test = test_set["Close"].shift(shift).dropna()
	x_scaler = MinMaxScaler()
	y_scaler = MinMaxScaler()
	X_train_scaled = x_scaler.fit_transform(X_train)
	y_train_scaled = y_scaler.fit_transform(np.array(y_train).reshape(-1,1))
	X_test_scaled = x_scaler.transform(X_test)
	y_test_scaled = y_scaler.transform(np.array(y_test).reshape(-1,1))
	pd.to_pickle(train_set, "train_sets/" + ticker_code + "_train_set.pkl")
	pd.to_pickle(test_set, "test_sets/" + ticker_code + "_test_set.pkl")
	pd.to_pickle(X_train, "sets/X_train/" + ticker_code + "_X_train.pkl")
	pd.to_pickle(X_test, "sets/X_test/" + ticker_code + "_X_test.pkl")
	pd.to_pickle(y_train, "sets/y_train/" + ticker_code + "_y_train.pkl")
	pd.to_pickle(y_test, "sets/y_test/" + ticker_code + "_y_test.pkl")
	pd.to_pickle(X_train_scaled, "sets/X_train/" + ticker_code + "_X_train_scaled.pkl")
	pd.to_pickle(X_test_scaled, "sets/X_test/" + ticker_code + "_X_test_scaled.pkl")
	pd.to_pickle(y_train_scaled, "sets/y_train/" + ticker_code + "_y_train_scaled.pkl")
	pd.to_pickle(y_test_scaled, "sets/y_test/" + ticker_code + "_y_test_scaled.pkl")


In [21]:
top_25 = ["AAPL", "MSFT", "AMZN", "TSLA", "GOOGL", "BRK.B", "UNH", "JNJ", "XOM", "JPM", "META", "V", "PG", "NVDA", "HD", "CVX", "LLY", "MA", "ABBV", "PFE", "MRK", "PEP", "BAC", "KO"]

for ticker_code in top_25:
	X_train_scaled = pd.read_pickle("sets/X_train/" + ticker_code + "_X_train_scaled.pkl")
	y_train_scaled = pd.read_pickle("sets/y_train/" + ticker_code + "_y_train_scaled.pkl")
	X_train_list = []
	for i in range(steps, len(X_train_scaled)):
		X_train_list.append(X_train_scaled[i-steps:i])
	X_train_list = np.array(X_train_list)
	model = Sequential()
	model.add(LSTM(nodes_per_layer, activation='relu', input_shape=(steps, X_train_list.shape[2]), return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=True))
	model.add(Dropout(dropout_rate))
	model.add(LSTM(nodes_per_layer, return_sequences=False))
	model.add(Dense(units=1, activation='relu'))
	model.compile(optimizer='adam', loss=loss_function)
	model.fit(X_train_list, y_train_scaled[:(steps * shift)], epochs=epochs, batch_size=batch_size)
	X_test_scaled = pd.read_pickle("sets/X_test/" + ticker_code + "_X_test_scaled.pkl")
	y_test_scaled = pd.read_pickle("sets/y_test/" + ticker_code + "_y_test_scaled.pkl")
	X_test_list = []
	for i in range(steps, len(X_test_scaled)):
		X_test_list.append(X_test_scaled[i-steps:i])
	score = model.evaluate(X_test_scaled, y_test_scaled)
	pd.to_pickle(model, "models/" + ticker_code + "_score_" + score + ".pkl")

Epoch 1/50
423/423 [==============================] - 47s 79ms/step - loss: 0.0021
Epoch 2/50
423/423 [==============================] - 34s 79ms/step - loss: 2.8410e-04
Epoch 3/50
423/423 [==============================] - 33s 78ms/step - loss: 3.4845e-04
Epoch 4/50
423/423 [==============================] - 33s 78ms/step - loss: 2.9085e-04
Epoch 5/50
423/423 [==============================] - 22s 53ms/step - loss: 2.9159e-04
Epoch 6/50
423/423 [==============================] - 22s 52ms/step - loss: 2.8098e-04
Epoch 7/50
423/423 [==============================] - 22s 52ms/step - loss: 3.1546e-04
Epoch 8/50
423/423 [==============================] - 22s 52ms/step - loss: 1.8351e-04
Epoch 9/50
423/423 [==============================] - 22s 52ms/step - loss: 1.7191e-04
Epoch 10/50
423/423 [==============================] - 22s 52ms/step - loss: 1.8645e-04
Epoch 11/50
423/423 [==============================] - 22s 52ms/step - loss: 3.0171e-04
Epoch 12/50
423/423 [========================

ValueError: in user code:

    File "d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\keras\engine\training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\keras\engine\training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\keras\engine\training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\keras\engine\training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\ProgramFiles\Anaconda\envs\final-project\lib\site-packages\keras\en